### Add description of the dataset
for instance a way to download it from kaggle

In [1]:
import pandas as pd

from topicViz.clean import HTMLCleaner
from topicViz.clean import TextProcessor
from topicViz.textinjupyter import markdown_scape_chars

In [2]:
data_df = pd.read_csv('data/train.csv', index_col='Id')
data_df.columns = [column.lower() for column in data_df.columns]
# data_df.head()

In [3]:
# sample row from the data
n_row = 10
print(data_df.iloc[0])
print('#'*60)
row_body = data_df.iloc[n_row]['body']
markdown_scape_chars(row_body)

title                      Java: Repeat Task Every Random Seconds
body            <p>I'm already familiar with repeating tasks e...
tags                                               <java><repeat>
creationdate                                  2016-01-01 00:21:59
y                                                        LQ_CLOSE
Name: 34552656, dtype: object
############################################################


```
'<p>I am new to Ubuntu (Linux tbh). I encountered problem with starting MongoDB service. I want to created web app using Ruby on Rails and MongoDB but mongo seems to fail to start.</p>\n\n<p>I followed this <a href="https://docs.mongodb.org/v3.0/tutorial/install-mongodb-on-ubuntu/">MongoDB installation on Ubuntu</a> all of installation went without problems until I got to <code>sudo service mongod start</code> </p>\n\n<p><strong>mongod.log</strong></p>\n\n<pre><code>2016-01-01T10:58:15.545+0000 I CONTROL  ***** SERVER RESTARTED *****\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] MongoDB starting : pid=3868 port=27017 dbpath=/var/lib/mongodb 64-bit host=damian-CX61-0NC-CX61-0ND-CX61-0NF-CX61-0NE\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] db version v3.0.8\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] git version: 83d8cc25e00e42856924d84e220fbe4a839e605d\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] build info: Linux ip-10-187-89-126 3.13.0-24-generic #46-Ubuntu SMP Thu Apr 10 19:11:08 UTC 2014 x86_64 BOOST_LIB_VERSION=1_49\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] allocator: tcmalloc\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] options: { config: "/etc/mongod.conf", net: { bindIp: "127.0.0.1", port: 27017 }, storage: { dbPath: "/var/lib/mongodb", journal: { enabled: true } }, systemLog: { destination: "file", logAppend: true, path: "/var/log/mongodb/mongod.log" } }\n2016-01-01T10:58:15.567+0000 E NETWORK  [initandlisten] Failed to unlink socket file /tmp/mongodb-27017.sock errno:1 Operation not permitted\n2016-01-01T10:58:15.567+0000 I -        [initandlisten] Fatal Assertion 28578\n2016-01-01T10:58:15.567+0000 I -        [initandlisten] \n\n***aborting after fassert() failure\n</code></pre>\n\n<p>So what I am doing wrong or its different issue?</p>\n'
```

#### Remove HTML from posts

In [4]:
# instantiate cleaner
cleaner = HTMLCleaner()
html_cleaned_df = data_df.copy()

In [5]:
# clean the body column
html_cleaned_df['body'] = html_cleaned_df['body'].apply(
    cleaner.remove_preformatted_text
)

html_cleaned_df['body'] = html_cleaned_df['body'].apply(
    cleaner.remove_code_blocks
)

html_cleaned_df['body'] = html_cleaned_df['body'].apply(
    cleaner.remove_hyperlinks
)

html_cleaned_df['body'] = html_cleaned_df['body'].apply(
    cleaner.clean_html
)

/home/edgar/gdrive/career/portfolio/topic-visualization/src/topicViz/clean.py:218: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, 'html.parser')
/home/edgar/.local/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [6]:
# clean the tags column
html_cleaned_df['tags'] = html_cleaned_df['tags'].apply(
    cleaner.clean_html
)

In [7]:
n_row = 10
row_body = data_df.iloc[n_row]['body']
markdown_scape_chars(row_body)
print('#'*60)
row_body = html_cleaned_df.iloc[n_row]['body']
markdown_scape_chars(row_body)


```
'<p>I am new to Ubuntu (Linux tbh). I encountered problem with starting MongoDB service. I want to created web app using Ruby on Rails and MongoDB but mongo seems to fail to start.</p>\n\n<p>I followed this <a href="https://docs.mongodb.org/v3.0/tutorial/install-mongodb-on-ubuntu/">MongoDB installation on Ubuntu</a> all of installation went without problems until I got to <code>sudo service mongod start</code> </p>\n\n<p><strong>mongod.log</strong></p>\n\n<pre><code>2016-01-01T10:58:15.545+0000 I CONTROL  ***** SERVER RESTARTED *****\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] MongoDB starting : pid=3868 port=27017 dbpath=/var/lib/mongodb 64-bit host=damian-CX61-0NC-CX61-0ND-CX61-0NF-CX61-0NE\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] db version v3.0.8\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] git version: 83d8cc25e00e42856924d84e220fbe4a839e605d\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] build info: Linux ip-10-187-89-126 3.13.0-24-generic #46-Ubuntu SMP Thu Apr 10 19:11:08 UTC 2014 x86_64 BOOST_LIB_VERSION=1_49\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] allocator: tcmalloc\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] options: { config: "/etc/mongod.conf", net: { bindIp: "127.0.0.1", port: 27017 }, storage: { dbPath: "/var/lib/mongodb", journal: { enabled: true } }, systemLog: { destination: "file", logAppend: true, path: "/var/log/mongodb/mongod.log" } }\n2016-01-01T10:58:15.567+0000 E NETWORK  [initandlisten] Failed to unlink socket file /tmp/mongodb-27017.sock errno:1 Operation not permitted\n2016-01-01T10:58:15.567+0000 I -        [initandlisten] Fatal Assertion 28578\n2016-01-01T10:58:15.567+0000 I -        [initandlisten] \n\n***aborting after fassert() failure\n</code></pre>\n\n<p>So what I am doing wrong or its different issue?</p>\n'
```

############################################################


```
'I am new to Ubuntu (Linux tbh). I encountered problem with starting MongoDB service. I want to created web app using Ruby on Rails and MongoDB but mongo seems to fail to start.\nI followed this  all of installation went without problems until I got to  \nmongod.log\nSo what I am doing wrong or its different issue?\n'
```

### Check special rows

In [8]:
# not clear why these rows are special??
# i = 4
# special_rows = [8166, 8778, 24599, 32425]
# n_row = special_rows[i]

# row_body = data_cleaned_df.iloc[n_row]['body']
# markdown_scape_chars(row_body)

#### Clean text

In [9]:
# instantiate text processor
text_processor = TextProcessor()
text_cleaned_df = html_cleaned_df.copy()

In [10]:
# clean the body column
text_cleaned_df['body'] = text_cleaned_df['body'].apply(
    text_processor.replace_newlines
)

text_cleaned_df['body'] = text_cleaned_df['body'].apply(
    text_processor.to_lowercase
)

text_cleaned_df['body'] = text_cleaned_df['body'].apply(
    text_processor.replace_dash_quotes
)

text_cleaned_df['body'] = text_cleaned_df['body'].apply(
    text_processor.replace_marks
)

text_cleaned_df['body'] = text_cleaned_df['body'].apply(
    text_processor.replace_punctuation
)

In [11]:
n_row = 10

row_body = data_df.iloc[n_row]['body']
markdown_scape_chars(row_body)

print('#'*60)

row_body = html_cleaned_df.iloc[n_row]['body']
markdown_scape_chars(row_body)

print('#'*60)

row_body = text_cleaned_df.iloc[n_row]['body']
markdown_scape_chars(row_body)


```
'<p>I am new to Ubuntu (Linux tbh). I encountered problem with starting MongoDB service. I want to created web app using Ruby on Rails and MongoDB but mongo seems to fail to start.</p>\n\n<p>I followed this <a href="https://docs.mongodb.org/v3.0/tutorial/install-mongodb-on-ubuntu/">MongoDB installation on Ubuntu</a> all of installation went without problems until I got to <code>sudo service mongod start</code> </p>\n\n<p><strong>mongod.log</strong></p>\n\n<pre><code>2016-01-01T10:58:15.545+0000 I CONTROL  ***** SERVER RESTARTED *****\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] MongoDB starting : pid=3868 port=27017 dbpath=/var/lib/mongodb 64-bit host=damian-CX61-0NC-CX61-0ND-CX61-0NF-CX61-0NE\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] db version v3.0.8\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] git version: 83d8cc25e00e42856924d84e220fbe4a839e605d\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] build info: Linux ip-10-187-89-126 3.13.0-24-generic #46-Ubuntu SMP Thu Apr 10 19:11:08 UTC 2014 x86_64 BOOST_LIB_VERSION=1_49\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] allocator: tcmalloc\n2016-01-01T10:58:15.548+0000 I CONTROL  [initandlisten] options: { config: "/etc/mongod.conf", net: { bindIp: "127.0.0.1", port: 27017 }, storage: { dbPath: "/var/lib/mongodb", journal: { enabled: true } }, systemLog: { destination: "file", logAppend: true, path: "/var/log/mongodb/mongod.log" } }\n2016-01-01T10:58:15.567+0000 E NETWORK  [initandlisten] Failed to unlink socket file /tmp/mongodb-27017.sock errno:1 Operation not permitted\n2016-01-01T10:58:15.567+0000 I -        [initandlisten] Fatal Assertion 28578\n2016-01-01T10:58:15.567+0000 I -        [initandlisten] \n\n***aborting after fassert() failure\n</code></pre>\n\n<p>So what I am doing wrong or its different issue?</p>\n'
```

############################################################


```
'I am new to Ubuntu (Linux tbh). I encountered problem with starting MongoDB service. I want to created web app using Ruby on Rails and MongoDB but mongo seems to fail to start.\nI followed this  all of installation went without problems until I got to  \nmongod.log\nSo what I am doing wrong or its different issue?\n'
```

############################################################


```
'i am new to ubuntu  linux tbh   i encountered problem with starting mongodb service  i want to created web app using ruby on rails and mongodb but mongo seems to fail to start  i followed this all of installation went without problems until i got to mongod log so what i am doing wrong or its different issue  '
```

### Save cleaned data

In [13]:
html_cleaned_df.to_csv('data/html_clean.csv')
text_cleaned_df.to_csv('data/text_html_clean.csv')

In [14]:
html_cleaned_df.columns

Index(['title', 'body', 'tags', 'creationdate', 'y'], dtype='object')